* Confounder 변수 
- 연구/실험 환경적 요인	: center_name, geo_location, geo_coord
- 생물학적 배경 변수 : sex, age
- 기술적/분석적 변수 : project_id, ENA-CHECKLIST, ENA-FIRST-PUBLIC, ENA-LAST-UPDATE, sequencing method, collection_date 


In [3]:
import pandas as pd

# 1. CSV 불러오기
df = pd.read_csv("all_column_delete10.csv")

df.info()

# 라벨 종류 확인
print(df["parkinson"].unique())

# 라벨 통일 (의미 기준: PD 관련 vs Control 관련)
df["parkinson"] = df["parkinson"].replace({
    "Parkinson disease": "PD",
    "PD": "PD",
    "yes": "PD",
    "gut patient": "PD",
    "Control": "Control",
    "no": "Control",
    "gut control": "Control"
})

# 혹시 모를 공백/대소문자 정리
df["parkinson"] = df["parkinson"].str.strip().str.capitalize()

# 최종 확인
print(df["parkinson"].value_counts(dropna=False))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1425 entries, 0 to 1424
Data columns (total 67 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   sample_id                                            1425 non-null   object 
 1   alias                                                1424 non-null   object 
 2   project_id                                           1425 non-null   object 
 3   center_name                                          1424 non-null   object 
 4   title                                                1366 non-null   object 
 5   parkinson                                            1249 non-null   object 
 6   organism                                             1424 non-null   object 
 7   ENA-CHECKLIST                                        324 non-null    object 
 8   collection_date                                      1224 non-null  

confounder 후보 연속형인지 범주형인지 확인 -> info 로 

counfounder 변수의 dtype 이 object라 나와있어서 정리 필요

[0] 라이브러리 import & 전처리

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency, f_oneway
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

[1] Confounder 후보 변수 지정

In [ ]:
# label은 반드시 string으로 통일
df["label"] = df["parkinson"].astype(str)

# confounder 후보들
categorical_confounders = [
    "center_name", "geo_location", "geo_coord", "sex", "project_id",
    "ENA-CHECKLIST", "ENA-FIRST-PUBLIC", "ENA-LAST-UPDATE", "sequencing method",
    "collection_date", "env_biome"
]

continuous_confounders = ["age"]


[2] 범주형 Confounder vs Label 분석 (crosstab + chi-square)

In [ ]:
for var in categorical_confounders:
    if var not in df.columns:
        print(f"[{var}] 존재하지 않음. 스킵")
        continue
    print(f"\n==== {var} ====")
    try:
        ct = pd.crosstab(df[var], df["label"], normalize="index")
        display(ct)

        # 카이제곱 검정
        chi2_table = pd.crosstab(df[var], df["label"])
        chi2, p, _, _ = chi2_contingency(chi2_table)
        print(f"Chi-square p-value: {p:.4f}")
    except Exception as e:
        print(f"오류 발생: {e}")


[3] 연속형 Confounder vs Label 분석 (boxplot + ANOVA)

In [ ]:
for var in continuous_confounders:
    if var not in df.columns:
        print(f"[{var}] 존재하지 않음. 스킵")
        continue
    print(f"\n==== {var} ====")

    # boxplot
    sns.boxplot(data=df, x="label", y=var)
    plt.title(f"{var} vs label")
    plt.show()

    # ANOVA
    groups = [df[df["label"] == l][var].dropna() for l in df["label"].unique()]
    if all(len(g) > 1 for g in groups):
        f_stat, p = f_oneway(*groups)
        print(f"ANOVA p-value: {p:.4f}")
    else:
        print("그룹 중 데이터 부족으로 ANOVA 불가")


[5] 도메인 해석 및 판단은 따로 정리